# <center> Monte Carlo Markov Chain for d-dimensional Ising Models </center>

## Assignment \# 1
We are going to construct a MCMC for an Ising model on a d-dimensional rectangular lattice with periodic boundary conditions. We will progress step-by-step. We first implement a *slow* version computing explicitely the energy of the Ising model with a function
```
energyslow(x::Lattice, h::Vector{Float64}, spin::Vector{Int})
```
The energy should return the value of the energy:
$$
H(\vec s) =  -\sum_{i=1}^N \left( \frac12 \sum_{j\in\partial_i} s_j + h_i\right)s_i
$$
where with the symbol $\partial_i$ we indicate the 2d set of neighbors of site $i$.

Defining `nsweep` a set of `N` elementary Monte Carlo steps (i.e. MC trials accepted or not), verify that the magnetization in a $(30\times 30)$ lattice with $h_i=0$ (no external fields)
$$
M := \frac1N \sum_{i=1}^N s_i 
$$
at inverse temperature $\beta=0.5$ after ~ 500 sweeps, ends fluctuating around a value 0.911319377877496.

In general we have that, from the exact Onsager solution valid at $d=2$ for h=0 

$$
M(\beta) = \left[ 1- \left(\frac{1}{\sinh(2\beta)}\right)^4\right]^{1/8}\quad,\quad \beta > \beta_c=\frac {\ln (1+\sqrt{2})}{2}
$$

## Assignment \# 2

Define a structure 

```
struct Ising{T<:AbstractFloat,D} 
    N::Int               # number of spin
    β::T                 # inverse temperature
    Λ::Lattice{D}        # Lattice
    spin::Vector{Int}    # spin config
    h::Vector{T}         # external field
    H::Vector{T}         # total field 
end
```

The total field `H[i]` is the local field acting on site `i` defined as:
$$
H_i = \sum_{j\in\partial_i} s_j + h_i \quad,\quad i\in 1,\dots,N
$$

The local field `H[i]` is useful in that:
1. we can see that if we flip spin `i` the energy variation is (show it!):
$$
\Delta E = E(\vec s_{-i}) - E(\vec s)= 2 H_i s_i
$$
where $s_{-i}$ is a shorthand for the spin configuration $s_1,\dots,-s_i,\dots,s_N$
2. The total energy can be written as (show it!):
$$
E = \sum_{i=1}^N -\frac12(H_i + h_i)s_i
$$

If  spin `i` is flipped (accepted change) the local field $H_j$ for $j\in\partial_i$ should be updated. How?

Detailed assignement:


* Write a constructor for `struct Ising` 

```
Ising(I::NTuple{D,Int},h::Vector{T},β::T, x0::Vector{Int})
```

where I is a `NTuple` containing `(Lx,Ly,Lz,...)` the external field `h`, the inverse temperature  `β`, and the initial spin configuration `x0`.

* Write a method 

```
function onemcstep!(x::Ising, site::Int)
```

performing the *tentative*  update of spin `site` according to the Metropolis-Hasting rule

* Write a method

```
function onemcsweep!(x::Ising)
```

which updates in a random order (use `randperm()` after a `using Random` statement) all spins

## A short note about constructors in Julia

Given a generic `struct T`, a default constructor is always defined:

In [1]:
struct T
    n::Int
    v::Vector{Int}
end

x = T(10,ones(Int,30));
x.n, x.v

(10, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

Given a generic `struct T`  we can always define a *custom* constructor for the struct as a method with the same name of the `struct` calling the default constructor

In [4]:
function T(n::Int)
    return T(n,ones(Int,n))
end

T(10)

T(10, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

The *custom* external constructor `T` is typically used for two main reasons: 

1. To make sanity checks on the input members of the class (e.g. checking for coherent size of arrays, etc.) before instantiating the `struct`.

2. To simplify the generation of complex `struct` where, for instance, only the knowledge of a part of the field is enough to generate the whole `struct`, as in the simple example above where the knowledge of the member `n` is used to generate the second member `v`.
